# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [ ]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from app.agent.utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output


# You can now use the imported functions
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

print("Successfully imported modules and set up the LLM client.")


# Load the application code from Day 3 to provide context for test generation
# app_code = load_artifact("app/main.py")
# if not app_code:
#     print("Warning: Could not load app/main.py. Lab may not function correctly.")

# Uses absolute path to avoid path issues in different environments
file_path = r"c:\Users\labadmin\Desktop\capstone\ai_enabled_eng_sw_capstone\app\main.py"
app_code = None
try:
    with open(file_path, "r") as f:
        app_code = f.read()
except FileNotFoundError:
    pass  # app_code remains None

if not app_code:
    print(f"Warning: Could not load {file_path}. Lab may not function correctly.")
else:
    print(f"Successfully loaded {file_path}.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4.1'
Successfully imported modules and set up the LLM client.
Successfully loaded c:\Users\labadmin\Desktop\capstone\ai_enabled_eng_sw_capstone\app\main.py.


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [3]:
happy_path_tests_prompt_base = f"""Provide a detailed prompt for creating happy path tests with python
adding a placeholder for the context needed for effective prompt"""


In [4]:
happy_path_prompt = get_completion(happy_path_tests_prompt_base, client, model_name, api_provider)
print(happy_path_prompt)

Certainly! Here’s a detailed prompt you can use (or give to an LLM) to generate **happy path tests in Python**. There’s a placeholder for you to add the **context** (such as a description of the function, code, or API to be tested):

---

**Prompt for Creating Happy Path Tests in Python**

You are an expert Python software tester. Please write a set of **happy path unit tests** in Python for the following context:

[INSERT CONTEXT HERE: e.g., function description, code snippet, API endpoint, class, etc.]

**Instructions:**

- Use the `unittest` or `pytest` framework (specify which one in your example).
- Focus on **happy path scenarios**: valid inputs, expected outputs, and standard (non-error) execution flows.
- For each test, explain what is being tested and why it’s a happy path.
- Make sure your test names are descriptive.
- If necessary, include any setup or teardown logic.
- Assume all dependencies are available.

**Example Context Placeholder:**
```python
# Example function to b

In [5]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
You are an expert Python developer and test engineer.  
Given the following context about the code/functionality to be tested, generate clear, self-contained happy path test cases
using Python and the `pytest` framework fpr the `POST /users/` endpoint.
Assert that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the 
response body).
Generate another test for the `GET /users/` endpoint.
Save the generated tests into a file named `tests/test_main_simple_1.py`
A happy path test checks that the code works as expected under normal, valid conditions (no errors or edge cases).

**Instructions:**  
- Write at least 3 happy path test cases covering typical valid scenarios.
- Use clear and descriptive test function/method names.
- Each test should be independently runnable.
- Assume all necessary imports (import the function to be tested and the test framework).
- Do not test error handling or edge cases unless they are part of the intended happy path.
- Include any setup or test data needed.
- Output only the code for the tests.

Use the app_code below as context for generating the tests.

"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple_1.py")
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# tests/test_main_simple_1.py

import pytest
from fastapi.testclient import TestClient
from main import app  # Assuming your FastAPI app is defined in main.py

client = TestClient(app)

def test_create_user_successful():
    """
    Test that a user is created successfully with valid data.
    """
    user_data = {
        "username": "john_doe",
        "email": "john@example.com",
        "full_name": "John Doe",
        "password": "securepassword123"
    }
    response = client.post("/users/", json=user_data)
    assert response.status_code in (200, 201)
    data = response.json()
    assert "id" in data
    assert data["username"] == user_data["username"]
    assert data["email"] == user_data["email"]
    assert data["full_name"] == user_data["full_name"]

def test_create_another_user_successful():
    """
    Test that a second user can be created successfully.
    """
    user_data = {
        "username": "jane_smith",
        "email": "jane@e

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [6]:
edge_case_tests_prompt_base = f"""Provide a detailed prompt for creating edge case tests with python
adding a placeholder for the context needed for an effective prompt"""


In [7]:
edge_case_prompt = get_completion(edge_case_tests_prompt_base, client, model_name, api_provider)
print(edge_case_prompt)

Certainly! Here is a detailed prompt template for generating edge case tests in Python, with a placeholder for you to add context about the function or system under test:

---

**Prompt Template for Edge Case Tests in Python**

> **Context:**  
> [Insert a clear and concise description of the function, method, or system under test. Include its purpose, expected inputs and outputs, constraints, and any relevant business rules or assumptions.]

> **Instructions:**  
> - Based on the above context, generate a comprehensive set of Python test cases focused on edge cases and boundary conditions.  
> - For each test, include a brief comment explaining why it is considered an edge case.  
> - Implement the tests using Python’s built-in `unittest` framework (or another testing library, if specified in the context).  
> - Ensure the tests are clear, concise, and self-contained.  
> - Cover all relevant input types, minimum and maximum values, empty inputs, invalid formats, and any other scenari

In [9]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
You are an expert Python developer and test engineer.
Use the following context about the code/functionality to be tested, generate clear, self-contained edge case test cases
using Python and the `pytest` framework for the `POST /users/` endpoint.  
Assert that invalid user data is handled correctly (e.g., checking for a `400 Bad Request` status code and verifying the 
response body contains appropriate error messages).
Generate another test for the `GET /users/` endpoint.
Save the generated tests into a file named `tests/test_main_edge_1.py`
An edge case test checks how the code handles unusual, extreme, or unexpected inputs or situations. 

Instructions:
{edge_case_prompt}


Use the app_code below as context for generating the tests.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
# File: tests/test_main_edge_1.py

import pytest
from fastapi.testclient import TestClient

# Import your FastAPI app here
# from app.main import app
# For demonstration, we'll assume `app` is imported

# Replace this import with your actual app
from app import app

client = TestClient(app)

# =========================
# POST /users/ edge case tests
# =========================

@pytest.mark.parametrize(
    "user_data, expected_error",
    [
        # Empty payload: No data at all
        ({}, "Missing required fields"),
        # Missing username
        ({"email": "test@example.com", "password": "password123"}, "username"),
        # Missing email
        ({"username": "testuser", "password": "password123"}, "email"),
        # Missing password
        ({"username": "testuser", "email": "test@example.com"}, "password"),
        # Empty username
        ({"username": "", "email": "test@example.com", "password": "password123"}, "username"),
        # 

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [ ]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
AS a expert Python developer and test engineer.
Using the context below about the code/functionality to be tested, generate a pytest fixture that sets up an isolated test database
using SQLite in-memory database for testing purposes. The fixture should create the database schema before each test and 
tear it down afterward.
Include any necessary imports and setup code.
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest_1.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
As an expert Python developer and test engineer.
Using the context below about the code/functionality to be tested, refactor the happy path tests to utilize the new pytest fixture for the isolated test database.
Ensure that the tests are still clear, self-contained, and independently runnable.
Include any necessary imports and setup code.
Save the refactored tests into a file named `tests/test_main_with_fixture_1.py`
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture_1.py")
else:
    print("Skipping test refactoring because app context is missing.")

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.